In [734]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pprint import pprint

In [735]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__,external_stylesheets=external_stylesheets, suppress_callback_exceptions=True)

mouse_data = pd.read_csv('C:/Users/S145/Desktop/2-nd term/Data_Visualization/Week 13. Practice Session/Mouse_metadata.csv')
study_results = pd.read_csv('C:/Users/S145/Desktop/2-nd term/Data_Visualization/Week 13. Practice Session/Study_results.csv')
merged_df = pd.merge(mouse_data, study_results, on = 'Mouse ID')

In [736]:
np.unique(mouse_data['Drug Regimen'])

array(['Capomulin', 'Ceftamin', 'Infubinol', 'Ketapril', 'Naftisol',
       'Placebo', 'Propriva', 'Ramicane', 'Stelasyn', 'Zoniferol'],
      dtype=object)

In [737]:
# -------------------------------------------------------------------------------------------------------------- 

#											PART 1: DESIGN PARAMETERS

# --------------------------------------------------------------------------------------------------------------
# Here we will set the colors, margins, DIV height&weight and other parameters

color_choices = {
	'light-blue': '#7FAB8',
	'light-grey': '#F7EFED',
	'light-red':  '#F1485B',
	'dark-blue':  '#33546D',
	'middle-blue': '#61D4E2'
}

drug_colors = {
	'Placebo':		'#29304E',
	'Capomulin':	'#27706B',	
	'Ramicane':		'#71AB7F',
	'Ceftamin':		'#9F4440',
	'Infubinol':	'#FFD37B',
	'Ketapril':		'#FEADB9',
	'Naftisol':		'#B3AB9E',
	'Propriva':		'#ED5CD4',
	'Stelasyn':		'#97C1DF',
	'Zoniferol':	'#8980D4'
}


colors = {
		'full-background':		color_choices['light-grey'],
		'chart-background':		color_choices['light-grey'],
		'histogram-color-1':	color_choices['dark-blue'],
		'histogram-color-2':	color_choices['light-red'],
		'block-borders':		color_choices['dark-blue']
}

margins = {
		'block-margins': '10px 10px 10px 10px',
		'block-margins': '4px 4px 4px 4px'
}

sizes = {
		'subblock-heights': '290px'
}

drug_groups = {
    'Lightweight': ['Ramicane', 'Capomulin'],
    'Heavyweight': ['Ceftamin', 'Infubinol', 'Ketapril', 'Naftisol','Propriva', 'Stelasyn', 'Zoniferol'],
    'Placebo': ['Placebo']
}

# -------------------------------------------------------------------------------------------------------------- 

#											PART 2: ACTUAL LAYOUT

# --------------------------------------------------------------------------------------------------------------
# Here we will set the DIV-s and other parts of our layout
# We need to have a 2x2 grid
# I have also included 1 more grid on top of others, where we will show the title of the app

app.layout = html.Div([
    html.Div(children =	html.H1('Mouse Data Study Results'),
					style ={
							'border': '3px {} solid'.format(colors['block-borders']),
							'margin': margins['block-margins'],
							'text-align': 'center'
							}
					),

    html.Div([
        html.Div([
            dcc.Checklist(
				id = 'weight-histogram-checklist',
		        options=[
		        	{'label': drug, 'value': drug} for drug in np.unique(mouse_data['Drug Regimen'])
		        ],
		        value=['Placebo'],
		        labelStyle={'display': 'inline-block'}
			),
            dcc.Graph(
				id = 'weight-histogram',
		        
			),
        ], style = {
							'border': '1px {} solid'.format(colors['block-borders']),
							'margin': margins['block-margins'],
							'width': '49%',
							# 'height': sizes['subblock-heights'],
					}
		),

        html.Div([
            dcc.RadioItems(
				id = 'all-histogram-RadioItems',
		        options=[{'label': drug, 'value': drug} for drug in np.unique(mouse_data['Drug Regimen'])],
		        value='Placebo',
		        labelStyle={'display': 'inline-block'}
			),
            dcc.Graph(
				id = 'all-histogram',
		        
			),
        ], style = {
							'border': '1px {} solid'.format(colors['block-borders']),
							'margin': margins['block-margins'],
							'width': '49%',
							# 'height': sizes['subblock-heights'],
					}
		)
    ], style={'display': 'flex', 'flex-wrap': 'wrap'}),

    html.Div([
        html.Div([
            dcc.Checklist(
				id = 'partial-histogram-checklist',
		        options=[{'label': key, 'value': key} for key in drug_groups.keys()],
        		value=['Placebo'],  # Default selection
        		labelStyle={'display': 'inline-block'}
			),
            html.Div(
                id='drug-checklist-container'),
            dcc.Graph(
				id = 'partial-histogram',
		        
			),
        ], style = {
							'border': '1px {} solid'.format(colors['block-borders']),
							'margin': margins['block-margins'],
							'width': '49%',
							# 'height': sizes['subblock-heights'],
					}
		),
        html.Div([
            dcc.Checklist(
				id = 'weight-line-checklist',
		        options=[{'label': key, 'value': key} for key in drug_groups.keys()],
        		value=['Placebo'],  # Default selection
		        labelStyle={'display': 'inline-block'}
			),
            dcc.Graph(
				id = 'weight-line',
		        
			),
        ], style = {
							'border': '1px {} solid'.format(colors['block-borders']),
							'margin': margins['block-margins'],
							'width': '49%',
							# 'height': sizes['subblock-heights'],
					}
		),
    ], style={'display': 'flex', 'flex-wrap': 'wrap'}),
], style={'backgroundColor': colors['full-background']})


# histogram of mice weights' for each drug
# it is a stacked histogram which lets us put histograms on top of each other 
@app.callback(
    Output(component_id='weight-histogram', component_property='figure'),
    [Input(component_id='weight-histogram-checklist', component_property='value')]
)

def update_weight_histogram(drug_names):
    
    traces = []

    for drug in drug_names:
    	traces.append(go.Histogram(x=mouse_data[mouse_data['Drug Regimen']==drug]['Weight (g)'],
    							name = drug,
    							opacity = 0.9,
    							marker = dict(color=drug_colors[drug]))
    				)

    return {
        'data': traces,
        'layout': dict(
        	barmode='stack',
            xaxis={'title': 'mouse weight',
   					'range': [merged_df['Weight (g)'].min(), merged_df['Weight (g)'].max()],
   					'showgrid': False
   					},
            yaxis={'title': 'number of mice', 
            		'showgrid': False,
            		'showticklabels': True
            		},
            autosize=False,
           	paper_bgcolor = colors['chart-background'],
           	plot_bgcolor = colors['chart-background'],
            margin={'l': 40, 'b': 40, 't': 10, 'r': 40},
	        legend={'x': 0, 'y': 1},
        )
    }
    
    
    
@app.callback(
     Output(component_id='all-histogram', component_property='figure'),
     [Input(component_id='all-histogram-RadioItems', component_property='value')])


def update_all_histogram(selected_drug):
    traces = []

    # Overall weight distribution
    overall_distribution = go.Histogram(x=mouse_data['Weight (g)'],
                                        name='All mice',
                                        opacity=0.7,
                                        marker=dict(color=drug_colors[selected_drug]))

    traces.append(overall_distribution)

    # Weight distribution for the selected drug
    if selected_drug:
        selected_distribution = go.Histogram(x=mouse_data[mouse_data['Drug Regimen'] == selected_drug]['Weight (g)'],
                                            name=selected_drug,
                                            opacity=0.9,
                                            marker=dict(color=drug_colors[selected_drug]))

        traces.append(selected_distribution)

    return {
        'data': traces,
        'layout': dict(
            barmode='overlay',
            xaxis={'title': 'Mouse Weight'},
            yaxis={'title': 'Number of Mice'},
            autosize=False,
            paper_bgcolor=colors['chart-background'],
            plot_bgcolor=colors['chart-background'],
            margin={'l': 40, 'b': 40, 't': 10, 'r': 40},
            legend={'x': 0, 'y': 1},
        )
    }

@app.callback(
    Output('drug-checklist-container', 'children'),
    [Input('partial-histogram-checklist', 'value')]
)
def update_drug_checklist_container(selected_groups):
    if not selected_groups:
        return None
    options = []
    for group in selected_groups:
        options.extend([{'label': drug, 'value': drug} for drug in drug_groups[group]])
    return dcc.Checklist(
        id='drug-checklist',
        options=options,
        value=[],  # Default no selection
        labelStyle={'display': 'inline-block'}
    )

@app.callback(
    Output('partial-histogram', 'figure'),
    [Input('drug-checklist', 'value')]
)
def update_partial_histogram(selected_drugs):
    if not selected_drugs:
        return {
            'data': [],
            'layout': go.Layout(
                title='No data selected',
                xaxis={'title': 'Mouse Weight'},
                yaxis={'title': 'Number of Mice'}
            )
        }
    traces = []
    for drug in selected_drugs:
        filtered_data = merged_df[merged_df['Drug Regimen'] == drug]
        traces.append(go.Histogram(
            x=filtered_data['Weight (g)'],
            name=drug,
            opacity=0.75,
            marker=dict(color=drug_colors.get(drug, '#333'))
        ))

    return {
        'data': traces,
        'layout': dict(
        	barmode='stack',
            xaxis={'title': 'Mouse weight',
   					'range': [merged_df['Weight (g)'].min(), merged_df['Weight (g)'].max()],
   					'showgrid': False
   					},
            yaxis={'title': 'Number of mice', 
            		'showgrid': False,
            		'showticklabels': True
            		},
            autosize=False,
           	paper_bgcolor = colors['chart-background'],
           	plot_bgcolor = colors['chart-background'],
            margin={'l': 40, 'b': 40, 't': 10, 'r': 40},
	        legend={'x': 0, 'y': 1},
        )
    }

@app.callback(
    Output(component_id='weight-line', component_property='figure'),
    [Input(component_id='weight-line-checklist', component_property='value')]
)

def update_weight_line(selected_groups):
    
    traces = []
    for group in selected_groups:
        for drug in drug_groups[group]:
            # Filter data for the drug
            drug_data = merged_df[merged_df['Drug Regimen'] == drug]
            # Group data by time and count the number of alive mice
            survival_data = drug_data.groupby('Timepoint').size().reset_index(name='Alive Mice')
            traces.append(go.Scatter(
                x=survival_data['Timepoint'],
                y=survival_data['Alive Mice'],
                mode='lines+markers',
                name=drug,
                marker=dict(color=drug_colors.get(drug, '#333'))  # Using default color if not specified
            ))

    return {
        'data': traces,
        'layout': dict(
        	barmode='stack',
            xaxis={'title': 'Time point',
   					'range': [merged_df['Timepoint'].min(), merged_df['Timepoint'].max()],
   					'showgrid': False
   					},
            yaxis={'title': 'Number of alive mice', 
            		'showgrid': False,
            		'showticklabels': True
            		},
            autosize=False,
           	paper_bgcolor = colors['chart-background'],
           	plot_bgcolor = colors['chart-background'],
            margin={'l': 40, 'b': 40, 't': 10, 'r': 150},
	        legend={'x': 1, 'y': 1},
        )
    }
    

In [738]:
# -------------------------------------------------------------------------------------------------------------- 

#											PART 4: RUNNING THE APP

# --------------------------------------------------------------------------------------------------------------
# >> use __ debug=True __ in order to be able to see the changes after refreshing the browser tab,
#			 don't forget to save this file before refreshing
# >> use __ port = 8081 __ or other number to be able to run several apps simultaneously
if __name__ == '__main__':
	app.run_server(debug=True, port = 8090
                #host = '0.0.0.0')
	)